# Stargate Tool

## A tool to bring data from Astra REST API.

In this demo we will create a LangChain Tool to query Astra using its REST API.

I will use data and structure from our Netflix Clone (https://github.com/DataStax-Examples/astra-netflix). 

I just had to create a new table with the PK being based on the Title column only.

In [88]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

# Loading Astra 

We will use the GraphQL to create the schema and load the data

In [96]:
import requests

def post_graphql_mutation(mutation_file_path, schema=False):
    
    with open(mutation_file_path, 'r') as file:
        graphql_mutation = file.read()

    payload = {
        'query': graphql_mutation
    }

    endpoint_url = f"https://{os.environ['STARGATE_DB_ID']}-{os.environ['STARGATE_DB_REGION']}.apps.astra.datastax.com/api/graphql"
    if schema:
        endpoint_url += '-schema'
    else:
        endpoint_url += f"/{os.environ['STARGATE_DB_KEYSPACE']}"
    headers = {
        'Content-Type': 'application/json',
        'x-cassandra-token': os.environ['STARGATE_DB_APPLICATION_TOKEN']
    }

    response = requests.post(endpoint_url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        response.raise_for_status()

In [91]:
# Creating Table
result = post_graphql_mutation("./movies_by_title_schema.gql",True)
print(result)

{'data': {'movies_by_title': True}}


In [97]:
# Inserting Data
result = post_graphql_mutation("./movies_by_title_data.gql")
print(result)

{'data': {'terminator2': {'value': {'title': 'Terminator 2'}}, 'inception': {'value': {'title': 'Inception'}}, 'prometheus': {'value': {'title': 'Prometheus'}}, 'aliens': {'value': {'title': 'Aliens'}}, 'bladeRunner': {'value': {'title': 'Blade Runner'}}, 'starTrek': {'value': {'title': 'Star Trek'}}, 'wonderWoman': {'value': {'title': 'Wonder Woman'}}, 'looper': {'value': {'title': 'Looper'}}, 'exMachina': {'value': {'title': 'Ex machina'}}, 'doctorStrange': {'value': {'title': 'Doctor Strange'}}, 'her': {'value': {'title': 'Her'}}, 'theMartian': {'value': {'title': 'The Martian'}}, 'deadpool': {'value': {'title': 'Deadpool'}}, 'upgrade': {'value': {'title': 'Upgrade'}}, 'okra': {'value': {'title': 'Okra'}}, 'xMen': {'value': {'title': 'X-men'}}, 'batmanBegins': {'value': {'title': 'Batman Begins'}}, 'ironMan': {'value': {'title': 'Iron Man'}}, 'kong': {'value': {'title': 'Kong: Skull Island'}}, 'aliens3': {'value': {'title': 'aliens 3'}}, 'avatar': {'value': {'title': 'Avatar'}}}}


# Creating the LangChain Tool

In [44]:
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [45]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [98]:
import requests

def stargate_get(table, pk):
    
    endpoint_url = f"https://{os.environ['STARGATE_DB_ID']}-{os.environ['STARGATE_DB_REGION']}.apps.astra.datastax.com/api/rest/v2/keyspaces/{os.environ['STARGATE_DB_KEYSPACE']}/{table}/{'/'.join(pk)}"
    
    headers = {
        'x-cassandra-token': os.environ['STARGATE_DB_APPLICATION_TOKEN']
    }
    
    response = requests.get(endpoint_url, headers = headers)
    
    res = response.json()
    
    if len(res['data']) > 0:
        return res['data'][0]
    else:
        return {}
    

In [75]:
def get_movie(movie_title):
    return stargate_get('movies_by_title',[movie_title])

In [76]:
# Testing with a movie
get_movie("Inception")

https://4dcd68d4-978d-45c6-a6b5-505416f98a61-us-east1.apps.astra.datastax.com/api/rest/v2/keyspaces/netflix1/movies_by_title/Inception


{'title': 'Inception',
 'year': 2010,
 'genre': 'Sci-Fi',
 'duration': 121,
 'synopsis': 'Cobb steals information from his targets by entering their dreams.',
 'thumbnail': 'https://i.imgur.com/RPa4UdO.mp4'}

In [77]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

Movie_tool = Tool(
    name='Stargate Movie DB',
    func= get_movie,
    description="Useful when a movie information is requested"
)

tools = [Movie_tool]

In [78]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [63]:
agent.run(
        "What about the movie Prometheus"
)



> Entering new AgentExecutor chain...
I should use the Stargate Movie DB to find information about the movie Prometheus.
Action: Stargate Movie DB
Action Input: Prometheushttps://4dcd68d4-978d-45c6-a6b5-505416f98a61-us-east1.apps.astra.datastax.com/api/rest/v2/keyspaces/netflix1/movies_by_genre/Sci-Fi/2012/Prometheus

Observation: {'genre': 'Sci-Fi', 'year': 2012, 'title': 'Prometheus', 'duration': 134, 'synopsis': "After a clue to mankind's origins is discovered, explorers are sent to the darkest corner of the universe.", 'thumbnail': 'https://i.imgur.com/L8k6Bau.mp4'}
Thought:I now know the final answer
Final Answer: Prometheus is a Sci-Fi movie released in 2012. It has a duration of 134 minutes and the synopsis is "After a clue to mankind's origins is discovered, explorers are sent to the darkest corner of the universe."

> Finished chain.


'Prometheus is a Sci-Fi movie released in 2012. It has a duration of 134 minutes and the synopsis is "After a clue to mankind\'s origins is discovered, explorers are sent to the darkest corner of the universe."'

In [67]:
agent.run(
        "When was the movie Inception released?"
)



> Entering new AgentExecutor chain...
I should use the Stargate Movie DB to find the release date of Inception.
Action: Stargate Movie DB
Action Input: Inceptionhttps://4dcd68d4-978d-45c6-a6b5-505416f98a61-us-east1.apps.astra.datastax.com/api/rest/v2/keyspaces/netflix1/movies_by_title/Inception

Observation: {'title': 'Inception', 'year': 2010, 'genre': 'Sci-Fi', 'duration': 121, 'synopsis': 'Cobb steals information from his targets by entering their dreams.', 'thumbnail': 'https://i.imgur.com/RPa4UdO.mp4'}
Thought:The release date of the movie Inception is 2010.
Final Answer: 2010

> Finished chain.


'2010'